Test Homework3

ชื่อ_รหัส 

1.ณัชณศา_6521

2.ชัญญาภัค_6567

Import Library

In [1]:
import roboticstoolbox as rtb
import numpy as np

from spatialmath import SE3
from math import pi
from HW3_utils import *
from FRA333_HW3_6521_6567 import *

ROBOT 3DOF

Parameter of Robot

In [2]:
d1 = 0.0892
a2 = -0.425
a3 = -0.39243
d4 = 0.109
d5 = 0.093
d6 = 0.082

DH Parameter (Modify)

In [3]:
T3_e = SE3(a3-d6,-d5,d4) * SE3.RPY(0,-pi/2,0)

robot = rtb.DHRobot(
    [
        rtb.RevoluteMDH(d= d1 ,offset= pi), 
        rtb.RevoluteMDH(alpha= pi/2), 
        rtb.RevoluteMDH(a= a2),
    ]
    ,tool = T3_e, name="3R robot")

print(robot)

DHRobot: 3R robot, 3 joints (RRR), dynamics, modified DH parameters
┌────────┬───────┬────────────┬────────┐
│  aⱼ₋₁  │ ⍺ⱼ₋₁  │     θⱼ     │   dⱼ   │
├────────┼───────┼────────────┼────────┤
│    0.0 │  0.0° │  q1 + 180° │ 0.0892 │
│    0.0 │ 90.0° │         q2 │    0.0 │
│ -0.425 │  0.0° │         q3 │    0.0 │
└────────┴───────┴────────────┴────────┘

┌──────┬─────────────────────────────────────────────────┐
│ tool │ t = -0.47, -0.093, 0.11; rpy/xyz = 0°, -90°, 0° │
└──────┴─────────────────────────────────────────────────┘



Prove1

In [5]:
def proofJacobian(q: list[float], robot: rtb.DHRobot) -> bool:
    # Jacobian from my answer 
    J_e = endEffectorJacobianHW3(q) 

    # Jacobian from Robotics Toolbox (reference base frame)
    J_ertb = robot.jacob0(q) 

    # Tolerance as needed
    allow_error = 0.0001  
    
    print(q)
    print("-----------Jacobian จากโค้ด-----------")
    print(J_e)
    print("-----------Jacob0 จาก Robotics Toolbox------------")
    print(J_ertb)
    print("------------Jacobian ถูกหรือไม่------------")
    
    # Compare Jacobian (my answer VS Robotics Toolbox)
    return np.allclose(J_e, J_ertb, atol=allow_error)

# Random q 
q = np.random.rand(3) * 2 * np.pi

result = proofJacobian(q, robot)

print("Jacobian is correct:", result)

[1.31023103 6.05960819 4.80956437]
-----------Jacobian จากโค้ด-----------
[[-4.59867799e-01  1.48547530e-01  1.24271195e-01]
 [ 9.80525517e-03  5.57136274e-01  4.66086448e-01]
 [ 0.00000000e+00 -4.46870836e-01 -3.24488341e-02]
 [ 0.00000000e+00 -9.66244519e-01 -9.66244519e-01]
 [ 0.00000000e+00  2.57626802e-01  2.57626802e-01]
 [ 1.00000000e+00  6.12323426e-17  6.12323426e-17]]
-----------Jacob0 จาก Robotics Toolbox------------
[[-4.59867799e-01  1.48547534e-01  1.24271199e-01]
 [ 9.80525517e-03  5.57136261e-01  4.66086438e-01]
 [ 0.00000000e+00 -4.46870826e-01 -3.24488320e-02]
 [-1.38777878e-17 -9.66244496e-01 -9.66244496e-01]
 [-4.16333634e-17  2.57626810e-01  2.57626810e-01]
 [ 1.00000000e+00  6.12323400e-17  6.12323400e-17]]
------------Jacobian ถูกหรือไม่------------
Jacobian is correct: True


Prove2

In [6]:
def proveSingularity(q: list[float], robot: rtb.DHRobot) -> bool:
    
    is_singular_HW3 = checkSingularityHW3(q)

    # Jacobian from Robotics Toolbox (Reference from base frame)
    J_ertb = robot.jacob0(q)  
    
    # part of linear velocity (3x3 matrix)
    J_v_part = J_ertb[:3, :] 
    
    # Determinant of Jacobian
    determinant = np.linalg.det(J_v_part)
    
    # Check Singularity from determinant
    is_singular_rtb = abs(determinant) < 1e-3

    print("q ที่สุ่มได้:", q)
    print("ค่า Determinant จาก RTB:", determinant)
    print("อยู่ในจุด Singularities จาก RTB:", is_singular_rtb)
    print("อยู่ในจุด Singularities จาก HW3:", is_singular_HW3)

    # Compare the results of the two functions  
    result = is_singular_HW3 == is_singular_rtb
    print("ผลลัพธ์ตรงกันหรือไม่:", result)
    
    return result

# random q for robot 3DOF
q = np.random.rand(3) * 2 * np.pi
print("ผลลัพธ์สุดท้าย:", proveSingularity(q,robot))

# example that Singularity 
# q = [5.88029104 1.70964541 3.03725032]

ไม่อยู่ในจุด Singularity
q ที่สุ่มได้: [0.02850424 0.80425769 1.48722653]
ค่า Determinant จาก RTB: -0.018003396568623157
อยู่ในจุด Singularities จาก RTB: False
อยู่ในจุด Singularities จาก HW3: False
ผลลัพธ์ตรงกันหรือไม่: True
ผลลัพธ์สุดท้าย: True


Prove3

In [3]:
def proofEffort(robot):
    # Generate random q and w
    q = np.random.rand(3) * 2 * np.pi
    # w is now an array of 6 elements (moment + force)
    w = np.random.uniform(-np.pi, np.pi, size=6)  

    # Separate force and moment
    f_e = w[3:]  # Force
    n_e = w[:3]  # Moment
    
    # Concatenate force and moment into wrench
    w_e = np.concatenate((f_e, n_e), axis=0)

    # Calculate joint effort using Robotics Toolbox
    a = robot.pay(W=w_e, q=q, frame=1)
    
    # Calculate joint effort using custom function
    b = computeEffortHW3(q, w)

    # Print the results
    print(f"Joint Effort RTB : {-a}")
    print(f"Joint Effort HW3 : {b}")
    
    # Compare the results with a tolerance
    if np.allclose(-a, b, atol=1e-4):  #tolerance as needed
        print("The results are correct and match!")
    else:
        print("The results do not match.")

In [ ]:
proofEffort(robot)